In [1]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import urllib.parse as parse
import httpagentparser
import ip2geotools
from ip2geotools.databases.noncommercial import DbIpCity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#importing the files
train_data_raw = pd.read_csv("../sample_data.csv")

In [3]:
train_data_raw.head()

,session id,uuid,timestamp,location,location path,location query,referral location,ua_string,geo_ip
0,3efae056-7cd2-4e89-9820-a1b938cbb746,9567f0c2-d71e-4ba3-afdc-d46e37894fb4,09/07/17 10:43,www.acmeservices.com,/editing-services/research-paper-editing-and-p...,keyword=edit%20paper&matchtype=e&device=c&crea...,www.google.com,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,73.132.164.124
1,7039c571-4487-4d96-8013-4b4b70e66dc7,NaN,09/07/17 10:44,www.acmeservices.com,/editing-services/english-language-editing-and...,keyword=professional%20editing%20services&matc...,www.google.com,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,100.35.108.133
2,5b31e12a-229f-4774-8dae-b7267d04cba2,9567f0c2-d71e-4ba3-afdc-d46e37894fb4,09/07/17 10:54,www.acmeservices.com,/editing-services/academic-editing-and-proofre...,keyword=academic%20proofreading&matchtype=p&de...,www.google.com,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,73.132.164.124
3,f072cc36-105d-4c55-b5b0-3247243d9ddd,NaN,09/07/17 11:04,www.acmeservices.com,/editing-services/english-language-check-servi...,keyword=language%20editing%20service&matchtype...,www.google.iq,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,159.255.163.4
4,3fff8ecf-7ec1-4345-a132-5d150a0b0f8c,a01ef48d-8533-4c82-93e5-c3d7c7f04d07,09/07/17 12:08,www.acmeservices.com,/editing-services/english-language-editing-and...,keyword=language%20editing&matchtype=p&device=...,www.google.dk,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2.110.98.148


In [4]:
# parse query into keyword, matchtype, device, creative and gclid
def parse_query_without_values_as_list(qsdata):
    return dict((k, v[0]) for k, v in parse.parse_qs(qsdata).items())

query_dict = dict(train_data_raw['location query'].map(parse_query_without_values_as_list))
query_df = pd.DataFrame(data=[*query_dict.values()], columns=['keyword','matchtype', 'device', 'creative', 'gclid'])
query_df.head()
train_data_raw_2 = pd.concat([train_data_raw, query_df], axis=1)
train_data_raw_2 = train_data_raw_2.drop(['location query'], axis=1)

In [5]:
train_data_raw_2.head()
#train_data_raw_2[:119]

,session id,uuid,timestamp,location,location path,referral location,ua_string,geo_ip,keyword,matchtype,device,creative,gclid
0,3efae056-7cd2-4e89-9820-a1b938cbb746,9567f0c2-d71e-4ba3-afdc-d46e37894fb4,09/07/17 10:43,www.acmeservices.com,/editing-services/research-paper-editing-and-p...,www.google.com,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,73.132.164.124,edit paper,e,c,201469956189,CMOHip-C_NQCFVJLDQod4RwGUA
1,7039c571-4487-4d96-8013-4b4b70e66dc7,NaN,09/07/17 10:44,www.acmeservices.com,/editing-services/english-language-editing-and...,www.google.com,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,100.35.108.133,professional editing services,e,c,201469956228,CjwKEAjwqIfLBRCk6vH_rJq7yD0SJACG18frd6-0DE5ER7...
2,5b31e12a-229f-4774-8dae-b7267d04cba2,9567f0c2-d71e-4ba3-afdc-d46e37894fb4,09/07/17 10:54,www.acmeservices.com,/editing-services/academic-editing-and-proofre...,www.google.com,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,73.132.164.124,academic proofreading,p,c,204838723656,CIXsrsyE_NQCFROHswodehQHkA
3,f072cc36-105d-4c55-b5b0-3247243d9ddd,NaN,09/07/17 11:04,www.acmeservices.com,/editing-services/english-language-check-servi...,www.google.iq,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,159.255.163.4,language editing service,p,c,201465365924,CjwKCAjwqIfLBRAkEiwAhtfH6_uus09gDdK39ITo102x-R...
4,3fff8ecf-7ec1-4345-a132-5d150a0b0f8c,a01ef48d-8533-4c82-93e5-c3d7c7f04d07,09/07/17 12:08,www.acmeservices.com,/editing-services/english-language-editing-and...,www.google.dk,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2.110.98.148,language editing,p,c,201456839030,CjwKEAjwqIfLBRCk6vH_rJq7yD0SJACG18fraU0P5DXYVq...


In [6]:
# parse ua string into os, browser and device(bot=True or bot=False)
def parse_ua_string(ua_str):
    tmp1 = httpagentparser.detect(ua_str)
    isBot = tmp1['bot']
    os = str(tmp1['platform']['name']) + " " + str(tmp1['platform']['version'])
    browser = str(tmp1['browser']['name'])
    if not (isBot):
        browser = browser + " " + str(tmp1['browser']['version'])
    result = [os , browser, isBot]
    return result

train_data_raw_2['os'], train_data_raw_2['browser'], train_data_raw_2['isBot'] = zip(*train_data_raw_2['ua_string'].map(parse_ua_string))
train_data_raw_3 = train_data_raw_2.drop(['ua_string'], axis=1)

In [7]:
train_data_raw_3.head()

,session id,uuid,timestamp,location,location path,referral location,geo_ip,keyword,matchtype,device,creative,gclid,os,browser,isBot
0,3efae056-7cd2-4e89-9820-a1b938cbb746,9567f0c2-d71e-4ba3-afdc-d46e37894fb4,09/07/17 10:43,www.acmeservices.com,/editing-services/research-paper-editing-and-p...,www.google.com,73.132.164.124,edit paper,e,c,201469956189,CMOHip-C_NQCFVJLDQod4RwGUA,Windows 10,Chrome 59.0.3071.115,False
1,7039c571-4487-4d96-8013-4b4b70e66dc7,NaN,09/07/17 10:44,www.acmeservices.com,/editing-services/english-language-editing-and...,www.google.com,100.35.108.133,professional editing services,e,c,201469956228,CjwKEAjwqIfLBRCk6vH_rJq7yD0SJACG18frd6-0DE5ER7...,Windows 10,Chrome 59.0.3071.115,False
2,5b31e12a-229f-4774-8dae-b7267d04cba2,9567f0c2-d71e-4ba3-afdc-d46e37894fb4,09/07/17 10:54,www.acmeservices.com,/editing-services/academic-editing-and-proofre...,www.google.com,73.132.164.124,academic proofreading,p,c,204838723656,CIXsrsyE_NQCFROHswodehQHkA,Windows 10,Chrome 59.0.3071.115,False
3,f072cc36-105d-4c55-b5b0-3247243d9ddd,NaN,09/07/17 11:04,www.acmeservices.com,/editing-services/english-language-check-servi...,www.google.iq,159.255.163.4,language editing service,p,c,201465365924,CjwKCAjwqIfLBRAkEiwAhtfH6_uus09gDdK39ITo102x-R...,Windows 8.1,Microsoft Internet Explorer 11.0,False
4,3fff8ecf-7ec1-4345-a132-5d150a0b0f8c,a01ef48d-8533-4c82-93e5-c3d7c7f04d07,09/07/17 12:08,www.acmeservices.com,/editing-services/english-language-editing-and...,www.google.dk,2.110.98.148,language editing,p,c,201456839030,CjwKEAjwqIfLBRCk6vH_rJq7yD0SJACG18fraU0P5DXYVq...,Windows 10,Chrome 59.0.3071.115,False


In [ ]:
#------------------------------------------------------------------------------
# shorter implementation of DbIpCity.get for this particular use
#def get_location(ip_address):
#    # process request
#    request = requests.get('http://api.db-ip.com/v2/' + 'free' + '/' + quote(ip_address), timeout=62)
#    # parse content
#    content = request.content.decode('utf-8')
#    content = json.loads(content)

    # prepare return value
#    ip_location = IpLocation(ip_address)

    # format data
#    ip_location.country = content['countryCode']
#    ip_location.city = content['city']
#    return ip_location
#------------------------------------------------------------------------------

# get country, city and region info from IP
def parse_ip_info(ip):
    response = DbIpCity.get(str(ip), api_key='free')
    country = response.country
#    region = response.region
    city = response.city
    result = [country , city]
    return result
#    print(response.ip_address)
#    print(response.city)
#    print(response.region)
#    print(response.country)
#    print(response.latitude)
#    print(response.longitude)
train_data_raw_3['country'], train_data_raw_3['city'] = zip(*train_data_raw_3['geo_ip'].map(parse_ip_info))
#train_data_raw_4 = train_data_raw_3.drop(['geo_ip'], axis=1)

In [8]:
train_data_raw_4 = train_data_raw_3.drop(['geo_ip'], axis=1)

In [9]:
train_data_raw_4.head()

,session id,uuid,timestamp,location,location path,referral location,keyword,matchtype,device,creative,gclid,os,browser,isBot
0,3efae056-7cd2-4e89-9820-a1b938cbb746,9567f0c2-d71e-4ba3-afdc-d46e37894fb4,09/07/17 10:43,www.acmeservices.com,/editing-services/research-paper-editing-and-p...,www.google.com,edit paper,e,c,201469956189,CMOHip-C_NQCFVJLDQod4RwGUA,Windows 10,Chrome 59.0.3071.115,False
1,7039c571-4487-4d96-8013-4b4b70e66dc7,NaN,09/07/17 10:44,www.acmeservices.com,/editing-services/english-language-editing-and...,www.google.com,professional editing services,e,c,201469956228,CjwKEAjwqIfLBRCk6vH_rJq7yD0SJACG18frd6-0DE5ER7...,Windows 10,Chrome 59.0.3071.115,False
2,5b31e12a-229f-4774-8dae-b7267d04cba2,9567f0c2-d71e-4ba3-afdc-d46e37894fb4,09/07/17 10:54,www.acmeservices.com,/editing-services/academic-editing-and-proofre...,www.google.com,academic proofreading,p,c,204838723656,CIXsrsyE_NQCFROHswodehQHkA,Windows 10,Chrome 59.0.3071.115,False
3,f072cc36-105d-4c55-b5b0-3247243d9ddd,NaN,09/07/17 11:04,www.acmeservices.com,/editing-services/english-language-check-servi...,www.google.iq,language editing service,p,c,201465365924,CjwKCAjwqIfLBRAkEiwAhtfH6_uus09gDdK39ITo102x-R...,Windows 8.1,Microsoft Internet Explorer 11.0,False
4,3fff8ecf-7ec1-4345-a132-5d150a0b0f8c,a01ef48d-8533-4c82-93e5-c3d7c7f04d07,09/07/17 12:08,www.acmeservices.com,/editing-services/english-language-editing-and...,www.google.dk,language editing,p,c,201456839030,CjwKEAjwqIfLBRCk6vH_rJq7yD0SJACG18fraU0P5DXYVq...,Windows 10,Chrome 59.0.3071.115,False


In [10]:
train_data_raw_4.describe()

,session id,uuid,timestamp,location,location path,referral location,keyword,matchtype,device,creative,gclid,os,browser,isBot
count,399,362,400,400,400,336,188,188,201,201,211,400,400,400
unique,390,320,386,4,39,52,44,3,3,54,205,29,50,2
top,e9b4b656-d27b-42f2-a6e5-047ddac6ba4e,bcfa630a-064f-429e-a048-def93ee048cc,14/07/17 7:49,www.acmeservices.jp,/,www.google.co.jp,acmeservices,e,c,138857680233,CjwKEAjwqIfLBRCk6vH_rJq7yD0SJACG18fr0Rf79K0YZO...,Windows 7,Chrome 59.0.3071.115,False
freq,4,5,3,100,231,74,85,111,179,46,2,153,144,390


In [11]:
train_data_raw_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 14 columns):
session id           399 non-null object
uuid                 362 non-null object
timestamp            400 non-null object
location             400 non-null object
location path        400 non-null object
referral location    336 non-null object
keyword              188 non-null object
matchtype            188 non-null object
device               201 non-null object
creative             201 non-null object
gclid                211 non-null object
os                   400 non-null object
browser              400 non-null object
isBot                400 non-null bool
dtypes: bool(1), object(13)
memory usage: 41.1+ KB


In [12]:
train_data_raw_4.isnull().sum(axis=0)/train_data_raw_4.shape[0]

session id           0.0025
uuid                 0.0950
timestamp            0.0000
location             0.0000
location path        0.0000
referral location    0.1600
keyword              0.5300
matchtype            0.5300
device               0.4975
creative             0.4975
gclid                0.4725
os                   0.0000
browser              0.0000
isBot                0.0000
dtype: float64

In [13]:
train_data_raw_4['session id'].fillna("None", inplace=True)
train_data_raw_4['uuid'].fillna("None", inplace=True)
train_data_raw_4['referral location'].fillna(train_data_raw_4['referral location'].value_counts().idxmax(), inplace=True)
train_data_raw_4['keyword'].fillna("None", inplace=True)
train_data_raw_4['gclid'].fillna("None", inplace=True)
train_data_raw_4['matchtype'].fillna(train_data_raw_4['matchtype'].value_counts().idxmax(), inplace=True)
train_data_raw_4['device'].fillna(train_data_raw_4['device'].value_counts().idxmax(), inplace=True)
train_data_raw_4['creative'].fillna(train_data_raw_4['creative'].value_counts().idxmax(), inplace=True)

In [14]:
train_data_raw_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 14 columns):
session id           400 non-null object
uuid                 400 non-null object
timestamp            400 non-null object
location             400 non-null object
location path        400 non-null object
referral location    400 non-null object
keyword              400 non-null object
matchtype            400 non-null object
device               400 non-null object
creative             400 non-null object
gclid                400 non-null object
os                   400 non-null object
browser              400 non-null object
isBot                400 non-null bool
dtypes: bool(1), object(13)
memory usage: 41.1+ KB


In [15]:
train_data_raw_4.describe()

,session id,uuid,timestamp,location,location path,referral location,keyword,matchtype,device,creative,gclid,os,browser,isBot
count,400,400,400,400,400,400,400,400,400,400,400,400,400,400
unique,391,321,386,4,39,52,45,3,3,54,206,29,50,2
top,e9b4b656-d27b-42f2-a6e5-047ddac6ba4e,None,14/07/17 7:49,www.acmeservices.jp,/,www.google.co.jp,None,e,c,138857680233,None,Windows 7,Chrome 59.0.3071.115,False
freq,4,38,3,100,231,138,212,323,378,245,189,153,144,390


In [16]:
train_data_raw_4.dtypes

session id           object
uuid                 object
timestamp            object
location             object
location path        object
referral location    object
keyword              object
matchtype            object
device               object
creative             object
gclid                object
os                   object
browser              object
isBot                  bool
dtype: object

In [17]:
train_data_raw_4['creative'] = pd.to_numeric(train_data_raw_4['creative'])

In [18]:
train_data_raw_4.dtypes

session id           object
uuid                 object
timestamp            object
location             object
location path        object
referral location    object
keyword              object
matchtype            object
device               object
creative              int64
gclid                object
os                   object
browser              object
isBot                  bool
dtype: object

In [19]:
#print(train_data_raw_4['timestamp'].value_counts())
print(train_data_raw_4['location'].value_counts())
print(train_data_raw_4['location path'].value_counts()[:12])
print(train_data_raw_4['referral location'].value_counts()[:11])

www.acmeservices.jp       100
www.acmeservices.com      100
www.acmeservices.co.kr    100
www.acmeservices.cn       100
Name: location, dtype: int64
/                                                                                       231
/editing-services/english-language-editing-and-proofreading-services.html                26
/ppc/english-editing/                                                                    26
/ppc/translation/                                                                        13
/editing-services/research-paper-editing-and-proofreading-services.html                   9
/publication-support/packs.html                                                           8
/translation/                                                                             8
/translation-services/chinese-simplified-and-traditional-to-english-translation.html      7
/insights/107-retracted-paper-follow-up-investigation-progress                            7
/translation-services/k

In [20]:
print(train_data_raw_4['keyword'].groupby(train_data_raw_4['location']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['matchtype'].groupby(train_data_raw_4['location']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['device'].groupby(train_data_raw_4['location']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['creative'].groupby(train_data_raw_4['location']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['gclid'].groupby(train_data_raw_4['location']).value_counts())

location                keyword                            
www.acmeservices.cn     None                                   100
www.acmeservices.co.kr  None                                    81
                        에디티지                                     8
                        acmeservices                             4
                        impact factor                            1
                        논문초록작성법                                  1
                        에디티지 코리아                                 1
                        영문교정                                     1
                        의학논문교정                                   1
                        의학번역                                     1
                        전문 번역                                    1
www.acmeservices.com    None                                    15
                        editing service                          9
                        acmeservices                             7
  

In [21]:
print(train_data_raw_4['keyword'].groupby(train_data_raw_4['timestamp']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['matchtype'].groupby(train_data_raw_4['timestamp']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['device'].groupby(train_data_raw_4['timestamp']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['creative'].groupby(train_data_raw_4['timestamp']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['gclid'].groupby(train_data_raw_4['timestamp']).value_counts())

timestamp       keyword     
02/08/17 11:46  None            1
02/08/17 12:14  None            1
02/08/17 12:15  None            1
02/08/17 12:19  None            1
02/08/17 12:23  None            2
02/08/17 12:26  None            1
02/08/17 12:40  None            2
02/08/17 12:43  None            1
02/08/17 12:46  None            1
02/08/17 12:51  None            2
02/08/17 12:58  None            1
02/08/17 13:00  None            1
02/08/17 13:18  None            1
02/08/17 13:19  None            1
02/08/17 13:21  None            2
02/08/17 13:23  None            1
02/08/17 13:24  None            1
02/08/17 13:25  None            1
02/08/17 13:28  None            1
02/08/17 13:34  None            1
02/08/17 13:44  None            1
02/08/17 13:51  None            1
02/08/17 14:01  None            1
02/08/17 14:02  None            1
02/08/17 14:10  None            1
02/08/17 14:17  None            1
02/08/17 14:30  None            1
02/08/17 14:36  None            1
02/08/17 14:46  Non

In [22]:
print(train_data_raw_4['keyword'].groupby(train_data_raw_4['referral location']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['matchtype'].groupby(train_data_raw_4['referral location']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['device'].groupby(train_data_raw_4['referral location']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['creative'].groupby(train_data_raw_4['referral location']).value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['gclid'].groupby(train_data_raw_4['referral location']).value_counts())

referral location                        keyword                            
ad.search.naver.com                      None                                    1
blog.sciencenet.cn                       None                                    1
bzclk.baidu.com                          None                                    2
cas.hk.as.criteo.com                     None                                    1
com.google.android.googlequicksearchbox  arabic%20to%20english%20translation     1
                                         english editing                         1
cpro.baidu.com                           None                                    3
display.ad.daum.net                      None                                    6
dp.g.doubleclick.net                     research paper editing                  2
googleads.g.doubleclick.net              None                                    3
lm.facebook.com                          None                                    2
m.baidu.co

In [23]:
print(train_data_raw_4['browser'].value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['os'].value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['device'].value_counts())

Chrome 59.0.3071.115                144
Microsoft Internet Explorer 11.0     73
Firefox 54.0                         29
Chrome 45.0.2454.101                 26
Chrome 50.0.2661.102                 18
Chrome 59.0.3071.125                 12
GoogleBot                            10
Chrome 49.0.2623.75                   6
Chrome 55.0.2883.87                   5
MSEdge 15.15063                       5
Chrome 49.0.2623.221                  5
Chrome 58.0.3029.110                  5
Chrome 53.0.2785.49                   5
Chrome 49.0.2623.112                  4
Chrome 30.0.0.0                       4
MSEdge 14.14393                       3
Chrome 55.0.2883.91                   3
Chrome 53.0.2785.104                  3
Chrome 31.0.1650.63                   3
Firefox 53.0                          2
Chrome 56.0.2924.87                   2
Microsoft Internet Explorer 10.0      2
Chrome 52.0.2743.116                  2
Chrome 33.0.0.0                       2
Chrome 50.0.2661.89                   2


In [ ]:
print(train_data_raw_4['country'].value_counts())
print("---------------------------------------------------------------------------------------------")
print(train_data_raw_4['city'].value_counts())